In [10]:
import sys

sys.path.append("/users/tom/git/neon_blink_detection/")
sys.path.append("/users/tom/git/neon_blink_detection/src")

from training.evaluation import evaluate
from training.helper import ClassifierParams, Results
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import cross_validate
from copy import copy
from functions.classifiers import Classifier, load_predictions, save_predictions
from src.post_processing import classify
of_params, pp_params, _ = get_params()

from src.neon_blink_detector import get_params
from src.features_calculator import create_grids
from pathlib import Path
import numpy as np
from random import choices
from training.datasets_loader import (
    concatenate,
    concatenate_all_samples,
)
from training.helper import (
    get_augmentation_options,
    get_export_dir,
    get_training_dir,
)
from src.event_array import Samples
from src.helper import OfParams, PPParams, AugParams
from training.helper import get_experiment_name_new
from training.video_loader import video_loader

clip_names = np.load("/users/tom/git/neon_blink_detection/clip_list.npy")

of_params = OfParams(n_layers=7, layer_interval=7, average=False, img_shape=(64, 64), grid_size=4, step_size=7, window_size=11, stop_steps=3)

aug_params_options = get_augmentation_options()
aug_params = aug_params_options[0]

rec = video_loader(of_params, aug_params)

rec.collect(clip_names, bg_ratio=1, augment=False, idx=0)

options 60

Loading clip: 1252-2023-01-26-09-38-13-5b1af4ce
Number of clips: 1

Loading clip: 1251-2023-01-25-16-37-59-6778eeb5
Number of clips: 1

Loading clip: 1249-2023-01-25-14-29-18-e54693e4
Number of clips: 1

Loading clip: 1248-2023-01-25-12-59-30-e6407072
Number of clips: 3

Loading clip: 1328-2023-02-06-09-45-58-f56cc84e
Number of clips: 2

Loading clip: 1327-2023-02-06-09-40-56-d5cf1243
Number of clips: 1

Loading clip: 1326-2023-02-03-17-45-57-8a41f628
Number of clips: 1

Loading clip: 1325-2023-02-03-17-46-35-11ad4b90
Number of clips: 1

Loading clip: 1322-2023-02-03-14-25-41-7883a14f
Number of clips: 2

Loading clip: 1318-2023-02-03-10-41-29-03c72c25
Number of clips: 1

Loading clip: 1317-2023-02-03-09-41-14-30f69b87
Number of clips: 2

Loading clip: 1316-2023-02-03-09-34-09-31ef7938
Number of clips: 4

Loading clip: 1311-2023-02-02-14-30-40-96be0928
Number of clips: 1

Loading clip: 1312-2023-02-02-16-21-47-cd28e8e0
Number of clips: 1

Loading clip: 1313-2023-02-02-16-31-

In [5]:
def compute_clf_scores(predictions, labels):

    pred = copy(predictions)
    pred[pred == 1] = 0
    pred[pred == 2] = 1
    lab = copy(labels)
    lab[lab == 1] = 0
    lab[lab == 2] = 1

    scores = {}

    scores["recall_off"] = recall_score(lab, pred)
    scores["precision_off"] = precision_score(lab, pred)
    scores["f1_off"] = f1_score(lab, pred)

    pred = copy(predictions)
    pred[pred == 2] = 0
    pred[pred == 1] = 1
    lab = copy(labels)
    lab[lab == 2] = 0
    lab[lab == 1] = 1

    scores["recall_on"] = recall_score(lab, pred)
    scores["precision_on"] = precision_score(lab, pred)
    scores["f1_on"] = f1_score(lab, pred)

    pred = copy(predictions)
    pred[pred > 0] = 1
    lab = copy(labels)
    lab[lab > 0] = 1

    scores["recall_all"] = recall_score(lab, pred)
    scores["precision_all"] = precision_score(lab, pred)
    scores["f1_all"] = f1_score(lab, pred)

    return scores


In [6]:
from functions.pipeline import get_classifier_params

classifier_params = get_classifier_params()

training_dir = get_training_dir(classifier_params.name, False)
export_dir = get_export_dir(classifier_params.name, False)

experiment_name = get_experiment_name_new(of_params, aug_params)
save_path = training_dir / experiment_name
export_path = export_dir / experiment_name

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)

features = concatenate(rec.all_features, clip_names)
samples_gt = concatenate_all_samples(rec.all_samples, clip_names)
labels = samples_gt.labels
X = features
y = labels

scores_train = []
scores_test = []
i = 0

for train, test in kf.split(X):

    print(i)

    x_train, x_test = X[train], X[test]
    y_train, y_test = y[train], y[test]

    classifier = Classifier(classifier_params, export_path)
    classifier.on_fit(x_train, y_train)

    predictions = classifier.predict(x_train)
    predictions = classify(predictions, pp_params)

    scores_train.append(f1_score(y_train, predictions, average=None))

    predictions = classifier.predict(x_test)
    predictions = classify(predictions, pp_params)

    scores_test.append(f1_score(y_test, predictions, average=None))

    i += 1


In [117]:
print("---------------- Training F1 scores ----------------")
print([f"Bg: {np.array(scores_train)[x, 0]:.2f}, " for x in range(0, 5)])
print([f"On: {np.array(scores_train)[x, 1]:.2f}, " for x in range(0, 5)])
print([f"Off: {np.array(scores_train)[x, 2]:.2f}, " for x in range(0, 5)])

print("---------------- Test F1 scores ----------------")
print([f"Bg: {np.array(scores_test)[x, 0]:.2f}, " for x in range(0, 5)])
print([f"On: {np.array(scores_test)[x, 1]:.2f}, " for x in range(0, 5)])
print([f"Off: {np.array(scores_test)[x, 2]:.2f}, " for x in range(0, 5)])

---------------- Training F1 scores ----------------
['Bg: 0.97, ', 'Bg: 0.98, ', 'Bg: 0.97, ', 'Bg: 0.97, ', 'Bg: 0.97, ']
['On: 0.91, ', 'On: 0.91, ', 'On: 0.92, ', 'On: 0.91, ', 'On: 0.91, ']
['Off: 0.91, ', 'Off: 0.92, ', 'Off: 0.91, ', 'Off: 0.91, ', 'Off: 0.91, ']
---------------- Test F1 scores ----------------
['Bg: 0.95, ', 'Bg: 0.94, ', 'Bg: 0.94, ', 'Bg: 0.95, ', 'Bg: 0.95, ']
['On: 0.81, ', 'On: 0.84, ', 'On: 0.79, ', 'On: 0.80, ', 'On: 0.83, ']
['Off: 0.85, ', 'Off: 0.82, ', 'Off: 0.83, ', 'Off: 0.83, ', 'Off: 0.82, ']


**Small training set (20%) and large validation set (80%)**

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)

features = concatenate(rec.all_features, clip_names)
samples_gt = concatenate_all_samples(rec.all_samples, clip_names)
labels = samples_gt.labels
X = features
y = labels

scores_train = []
scores_test = []
i = 0

for test, train in kf.split(X):

    print(i)

    x_train, x_test = X[train], X[test]
    y_train, y_test = y[train], y[test]

    classifier = Classifier(classifier_params, export_path)
    classifier.on_fit(x_train, y_train)

    predictions = classifier.predict(x_train)
    predictions = classify(predictions, pp_params)

    scores_train.append(f1_score(y_train, predictions, average=None))

    predictions = classifier.predict(x_test)
    predictions = classify(predictions, pp_params)

    scores_test.append(f1_score(y_test, predictions, average=None))

    i += 1



**Scores for small training (20%) and large validation set**

In [114]:
print("---------------- Training F1 scores ----------------")
print([f"Bg: {np.array(scores_train)[x, 0]:.2f}, " for x in range(0, 5)])
print([f"On: {np.array(scores_train)[x, 1]:.2f}, " for x in range(0, 5)])
print([f"Off: {np.array(scores_train)[x, 2]:.2f}, " for x in range(0, 5)])

print("---------------- Test F1 scores ----------------")
print([f"Bg: {np.array(scores_test)[x, 0]:.2f}, " for x in range(0, 5)])
print([f"On: {np.array(scores_test)[x, 1]:.2f}, " for x in range(0, 5)])
print([f"Off: {np.array(scores_test)[x, 2]:.2f}, " for x in range(0, 5)])


---------------- Training F1 scores ----------------
['Bg: 1.00, ', 'Bg: 0.99, ', 'Bg: 0.99, ', 'Bg: 0.99, ', 'Bg: 1.00, ']
['On: 0.98, ', 'On: 0.98, ', 'On: 0.98, ', 'On: 0.98, ', 'On: 0.99, ']
['Off: 0.98, ', 'Off: 0.98, ', 'Off: 0.98, ', 'Off: 0.98, ', 'Off: 0.99, ']
---------------- Test F1 scores ----------------
['Bg: 0.94, ', 'Bg: 0.93, ', 'Bg: 0.94, ', 'Bg: 0.92, ', 'Bg: 0.93, ']
['On: 0.76, ', 'On: 0.73, ', 'On: 0.81, ', 'On: 0.77, ', 'On: 0.74, ']
['Off: 0.78, ', 'Off: 0.73, ', 'Off: 0.82, ', 'Off: 0.78, ', 'Off: 0.78, ']


**Even smaller training (10%) and larger validation set (90%)**

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=10)

features = concatenate(rec.all_features, clip_names)
samples_gt = concatenate_all_samples(rec.all_samples, clip_names)
labels = samples_gt.labels
X = features
y = labels

scores_train = []
scores_test = []
i = 0

for test, train in kf.split(X):

    print(i)

    x_train, x_test = X[train], X[test]
    y_train, y_test = y[train], y[test]

    classifier = Classifier(classifier_params, export_path)
    classifier.on_fit(x_train, y_train)

    predictions = classifier.predict(x_train)
    predictions = classify(predictions, pp_params)

    scores_train.append(f1_score(y_train, predictions, average=None))

    predictions = classifier.predict(x_test)
    predictions = classify(predictions, pp_params)

    scores_test.append(f1_score(y_test, predictions, average=None))

    i += 1

In [120]:
print("---------------- Training F1 scores ----------------")
print([f"Bg: {np.array(scores_train)[x, 0]:.2f}, " for x in range(0, 10)])
print([f"On: {np.array(scores_train)[x, 1]:.2f}, " for x in range(0, 10)])
print([f"Off: {np.array(scores_train)[x, 2]:.2f}, " for x in range(0, 10)])

print("---------------- Test F1 scores ----------------")
print([f"Bg: {np.array(scores_test)[x, 0]:.2f}, " for x in range(0, 10)])
print([f"On: {np.array(scores_test)[x, 1]:.2f}, " for x in range(0, 10)])
print([f"Off: {np.array(scores_test)[x, 2]:.2f}, " for x in range(0, 10)])


---------------- Training F1 scores ----------------
['Bg: 1.00, ', 'Bg: 1.00, ', 'Bg: 1.00, ', 'Bg: 1.00, ', 'Bg: 1.00, ', 'Bg: 1.00, ', 'Bg: 1.00, ', 'Bg: 1.00, ', 'Bg: 1.00, ', 'Bg: 1.00, ']
['On: 1.00, ', 'On: 1.00, ', 'On: 1.00, ', 'On: 1.00, ', 'On: 1.00, ', 'On: 1.00, ', 'On: 1.00, ', 'On: 1.00, ', 'On: 1.00, ', 'On: 1.00, ']
['Off: 1.00, ', 'Off: 1.00, ', 'Off: 1.00, ', 'Off: 1.00, ', 'Off: 1.00, ', 'Off: 1.00, ', 'Off: 1.00, ', 'Off: 1.00, ', 'Off: 1.00, ', 'Off: 1.00, ']
---------------- Test F1 scores ----------------
['Bg: 0.93, ', 'Bg: 0.93, ', 'Bg: 0.93, ', 'Bg: 0.93, ', 'Bg: 0.93, ', 'Bg: 0.94, ', 'Bg: 0.93, ', 'Bg: 0.92, ', 'Bg: 0.93, ', 'Bg: 0.93, ']
['On: 0.73, ', 'On: 0.71, ', 'On: 0.73, ', 'On: 0.73, ', 'On: 0.73, ', 'On: 0.79, ', 'On: 0.76, ', 'On: 0.74, ', 'On: 0.70, ', 'On: 0.74, ']
['Off: 0.75, ', 'Off: 0.75, ', 'Off: 0.73, ', 'Off: 0.74, ', 'Off: 0.76, ', 'Off: 0.81, ', 'Off: 0.76, ', 'Off: 0.76, ', 'Off: 0.77, ', 'Off: 0.76, ']


In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=2000)

features = concatenate(rec.all_features, clip_names)
samples_gt = concatenate_all_samples(rec.all_samples, clip_names)
labels = samples_gt.labels
X = features
y = labels

scores_train = []
scores_test = []
i = 0

for test, train in kf.split(X, y):

    print(i)

    x_train, x_test = X[train], X[test]
    y_train, y_test = y[train], y[test]

    classifier = Classifier(classifier_params, export_path)
    classifier.on_fit(x_train, y_train)

    predictions = classifier.predict(x_train)
    predictions = classify(predictions, pp_params)

    scores_train.append(f1_score(y_train, predictions, average=None))

    predictions = classifier.predict(x_test)
    predictions = classify(predictions, pp_params)

    scores_test.append(f1_score(y_test, predictions, average=None))

    i += 1

In [73]:
print("---------------- Training F1 scores ----------------")
print([f"Bg: {np.array(scores_train)[x, 0]:.2f}, " for x in range(0, 10)])
print([f"On: {np.array(scores_train)[x, 1]:.2f}, " for x in range(0, 10)])
print([f"Off: {np.array(scores_train)[x, 2]:.2f}, " for x in range(0, 10)])

print("---------------- Test F1 scores ----------------")
print([f"Bg: {np.array(scores_test)[x, 0]:.2f}, " for x in range(0, 10)])
print([f"On: {np.array(scores_test)[x, 1]:.2f}, " for x in range(0, 10)])
print([f"Off: {np.array(scores_test)[x, 2]:.2f}, " for x in range(0, 10)])

---------------- Training F1 scores ----------------
['Bg: 1.00, ', 'Bg: 1.00, ', 'Bg: 1.00, ', 'Bg: 1.00, ', 'Bg: 1.00, ', 'Bg: 1.00, ', 'Bg: 1.00, ', 'Bg: 1.00, ', 'Bg: 1.00, ', 'Bg: 1.00, ']
['On: 1.00, ', 'On: 1.00, ', 'On: 1.00, ', 'On: 1.00, ', 'On: 1.00, ', 'On: 1.00, ', 'On: 1.00, ', 'On: 1.00, ', 'On: 1.00, ', 'On: 1.00, ']
['Off: 1.00, ', 'Off: 1.00, ', 'Off: 1.00, ', 'Off: 1.00, ', 'Off: 1.00, ', 'Off: 1.00, ', 'Off: 1.00, ', 'Off: 1.00, ', 'Off: 1.00, ', 'Off: 1.00, ']
---------------- Test F1 scores ----------------
['Bg: 0.86, ', 'Bg: 0.87, ', 'Bg: 0.74, ', 'Bg: 0.87, ', 'Bg: 0.85, ', 'Bg: 0.83, ', 'Bg: 0.82, ', 'Bg: 0.87, ', 'Bg: 0.82, ', 'Bg: 0.87, ']
['On: 0.37, ', 'On: 0.33, ', 'On: 0.39, ', 'On: 0.43, ', 'On: 0.36, ', 'On: 0.12, ', 'On: 0.34, ', 'On: 0.32, ', 'On: 0.37, ', 'On: 0.37, ']
['Off: 0.57, ', 'Off: 0.45, ', 'Off: 0.50, ', 'Off: 0.50, ', 'Off: 0.50, ', 'Off: 0.33, ', 'Off: 0.48, ', 'Off: 0.41, ', 'Off: 0.45, ', 'Off: 0.37, ']


In [51]:
scores_test

[0.7592609023557074,
 0.7562552681779868,
 0.6243068186859501,
 0.7716494388004081,
 0.739008917084424,
 0.6790847788474336,
 0.7069007586176301,
 0.7363360099374473,
 0.6994698549310145,
 0.7573089037753427,
 0.6759793265604898,
 0.5673772237256555,
 0.7028082161394792,
 0.6808482321103766,
 0.6589104298833237,
 0.6673284237611463]

In [63]:
classifier = Classifier(classifier_params, export_path)
classifier.on_fit(x_train, y_train)

predictions = classifier.predict(x_test)
predictions = classify(predictions, pp_params)

on_fit took 166 ms


In [79]:
predictions[0:500]-y_test[0:500]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  1,  1,  0,  0,  1,  0,  0,  0,  1,  1,  1,  1,  1,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0, -1,  1,  1,  1,  1,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0, -1, -1, -1

In [76]:
y_test[0:500]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,

In [66]:
len(train)

46